In [1]:
import autogen

config_list = [{
    "model":"mistral-small",
    "base_url":'http://localhost:11434/v1',
    "api_key":"ollama"
}]

llm_config = {
    # "request_timeout":800,
    "config_list":config_list
}

In [4]:
import os
import pickle
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type

# Request all access (permission to read/send/receive emails, manage the inbox, and more)
SCOPES = ['https://mail.google.com/']
your_email = input("Enter your email: ")

In [10]:
def gmail_authenticate():
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

In [5]:
def search_messages(service, query):
    result = service.users().messages().list(userId='me',q=query).execute()
    messages = [ ]
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
    return messages

In [6]:
# utility functions
def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [21]:
def parse_parts(service, parts, folder_name, message):
    """
    Utility function that parses the content of an email partition
    """
    if parts:
        for part in parts:
            filename = part.get("filename")
            mimeType = part.get("mimeType")
            body = part.get("body")
            data = body.get("data")
            # file_size = body.get("size")
            # part_headers = part.get("headers")
            if part.get("parts"):
                # recursively call this function when we see that a part
                # has parts inside
                parse_parts(service, part.get("parts"), folder_name, message)
            if mimeType == "text/plain":
                # if the email part is text plain
                if data:
                    text = urlsafe_b64decode(data).decode()
                    print(text)
            # elif mimeType == "text/html":
            #     # if the email part is an HTML content
            #     # save the HTML file and optionally open it in the browser
            #     if not filename:
            #         filename = "index.html"
            #     filepath = os.path.join(folder_name, filename)
            #     print("Saving HTML to", filepath)
            #     with open(filepath, "wb") as f:
            #         f.write(urlsafe_b64decode(data))
            # else:
            #     # attachment other than a plain text or HTML
            #     for part_header in part_headers:
            #         part_header_name = part_header.get("name")
            #         part_header_value = part_header.get("value")
            #         if part_header_name == "Content-Disposition":
            #             if "attachment" in part_header_value:
            #                 # we get the attachment ID 
            #                 # and make another request to get the attachment itself
            #                 print("Saving the file:", filename, "size:", get_size_format(file_size))
            #                 attachment_id = body.get("attachmentId")
            #                 attachment = service.users().messages() \
            #                             .attachments().get(id=attachment_id, userId='me', messageId=message['id']).execute()
            #                 data = attachment.get("data")
            #                 filepath = os.path.join(folder_name, filename)
            #                 if data:
            #                     with open(filepath, "wb") as f:
            #                         f.write(urlsafe_b64decode(data))

In [25]:
def read_message(service, message):
    """
    This function takes Gmail API `service` and the given `message_id` and does the following:
        - Downloads the content of the email
        - Prints email basic information (To, From, Subject & Date) and plain/text parts
    """
    msg = service.users().messages().get(userId='me', id=message['id'], format='full').execute()
    # parts can be the message body, or attachments
    payload = msg['payload']
    headers = payload.get("headers")
    parts = payload.get("parts")
    folder_name = "email"
    has_subject = False
    if headers:
        # this section prints email basic info & creates a folder for the email
        for header in headers:
            name = header.get("name")
            value = header.get("value")
            if name.lower() == 'from':
                # we print the From address
                print("From:", value)
            if name.lower() == "to":
                # we print the To address
                print("To:", value)
            if name.lower() == "subject":
            #     # make our boolean True, the email has "subject"
            #     has_subject = True
            #     # make a directory with the name of the subject
                # folder_name = clean(value)
            #     # we will also handle emails with the same subject name
            #     folder_counter = 0
            #     while os.path.isdir(folder_name):
            #         folder_counter += 1
            #         # we have the same folder name, add a number next to it
            #         if folder_name[-1].isdigit() and folder_name[-2] == "_":
            #             folder_name = f"{folder_name[:-2]}_{folder_counter}"
            #         elif folder_name[-2:].isdigit() and folder_name[-3] == "_":
            #             folder_name = f"{folder_name[:-3]}_{folder_counter}"
            #         else:
            #             folder_name = f"{folder_name}_{folder_counter}"
            #     os.mkdir(folder_name)
                print("Subject:", value)
            if name.lower() == "date":
                # we print the date when the message was sent
                print("Date:", value)
    if not has_subject:
        # if the email does not have a subject, then make a folder with "email" name
        # since folders are created based on subjects
        # if not os.path.isdir(folder_name):
        #     os.mkdir(folder_name)
        pass
    parse_parts(service, parts, folder_name, message)
    print("="*50)

In [26]:
results = search_messages(service, 'from: CVS')

In [27]:
for msg in results:
    read_message(service, msg)

From: CVS Pharmacy <info@alerts.cvs.com>
To: <ericbzhou0513@gmail.com>
Subject: With CVS, You're One step closer to a Better Tomorrow!
Date: Fri, 04 Oct 2024 09:39:30 -0600


Thank you for trusting CVS(R) to be with you for this journey.

HTTPGetWrap|https://www.cvs.com/pharmacy/v2/?DisableWebtrends=Y&DisableBluecore=Y#/pharmacy&cid=em_postcovax_thankyou_top_banner 
Helping bring you one step closer to a healthier tomorrow.

HTTPGetWrap|https://www.cvs.com/pharmacy/v2/?Desktop-image=Trigger1_Hero Image Border Module_1_Dk_Mb.jpg#/pharmacy 

We&rsquo;re honored that you came to CVS(R) for your vaccination.

Congratulations on being one step closer to a healthier future. 

It means so much to all of us 
at CVS that you trusted us as 
a partner 
on your health care journey.

&zwnj;

We&rsquo;re here to support you by offering easy and safe ways to keep
? you on the path to better health. Whether it&rsquo;s an allergy 
or an earache, 
a telehealth visit 
or same-day delivery of a new prescr

# Usesful Claude system prompt snippets

The assistant is Claude, created by Anthropic. The current date is {}. Claude’s knowledge base was last updated on April 2024. It answers questions about events prior to and after April 2024 the way a highly informed individual in April 2024 would if they were talking to someone from the above date, and can let the human know this when relevant. If asked about purported events or news stories that may have happened after its cutoff date, Claude never claims they are unverified or rumors. It just informs the human about its cutoff date. Claude cannot open URLs, links, or videos. If it seems like the user is expecting Claude to do so, it clarifies the situation and asks the human to paste the relevant text or image content directly into the conversation. If it is asked to assist with tasks involving the expression of views held by a significant number of people, Claude provides assistance with the task regardless of its own views. If asked about controversial topics, it tries to provide careful thoughts and clear information. It presents the requested information without explicitly saying that the topic is sensitive, and without claiming to be presenting objective facts. When presented with a math problem, logic problem, or other problem benefiting from systematic thinking, Claude thinks through it step by step before giving its final answer.

If the user asks for a very long task that cannot be completed in a single response, Claude offers to do the task piecemeal and get feedback from the user as it completes each part of the task.

Claude provides thorough responses to more complex and open-ended questions or to anything where a long response is requested, but concise responses to simpler questions and tasks. All else being equal, it tries to give the most correct and concise answer it can to the user’s message. Rather than giving a long response, it gives a concise response and offers to elaborate if further information may be helpful.

Claude is happy to help with analysis, question answering, math, coding, creative writing, teaching, role-play, general discussion, and all sorts of other tasks.

Claude responds directly to all human messages without unnecessary affirmations or filler phrases like “Certainly!”, “Of course!”, “Absolutely!”, “Great!”, “Sure!”, etc. Specifically, Claude avoids starting responses with the word “Certainly” in any way.

# Gmail advanced search query

https://support.google.com/mail/answer/7190

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Provides filtering criteria for the email search agent.",
    code_execution_config=False,
)

email_search = autogen.AssistantAgent(
    name="Email Search Agent",
    llm_config=llm_config,
    system_message="""
        An agent that searches for emails based on the user's criteria.
        The user can specify the sender, subject, and other criteria to search for emails.
        The agent will then search for emails that match the criteria and return the results to the user.
        The user can then read the emails and take further action if needed.
    """,
)